In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
#pm.help()
#pm.reset()
#pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df_jo = pm.df('df_jo')
#df_jo.info()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
df_jo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32092 entries, 0 to 32091
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id_cr                       32092 non-null  int64          
 1   id_fe                       32092 non-null  int64          
 2   fe_cr_id                    21055 non-null  float64        
 3   user_id                     32092 non-null  int64          
 4   active                      32092 non-null  int64          
 5   created_at                  32092 non-null  datetime64[ns] 
 6   created_at_fe               21055 non-null  datetime64[ns] 
 7   amount                      32092 non-null  float64        
 8   fee                         21055 non-null  float64        
 9   stat_cr                     32092 non-null  object         
 10  stat_fe                     32092 non-null  object         
 11  transfer_type               32092 non-nul

In [2]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            # #total_paid_fees=('fee', 'sum'),            
            # #Num_Solicitudes=('id_cr', 'nunique')
            # #Num_Solicitudes=('id_cr', lambda x: x.unique().sum())
            # Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())

            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
0,47,2020-04,0.0,100.0,1,0.000000,2020-10
1,47,2020-05,5.0,10.0,1,50.000000,2020-10
2,47,2020-08,10.0,10.0,1,100.000000,2020-10
3,47,2020-09,5.0,5.0,1,100.000000,2020-10
4,47,2020-10,10.0,6.0,2,166.666667,2020-10


In [6]:
df = pm.df('df_jo')
timeFormat ='s' #D
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))
df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)
df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]

user_id = 10116 #21465 (el mateix user_id te dos categories diferents al good user) #14631 (totes malament) #19655 (la primera malament despres tot be) 
#display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])
pd.options.display.max_columns = None
display(df_jo[df_jo.user_id == user_id])



,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status,updated_at,reco_creation,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,created_at_slot,created_at_slot_h,created_at_dow,good_user
2209,2262,0,NaN,10116,1,2020-05-28 22:05:04.483048,NaT,100.0,NaN,rejected,cr_regular,regular,nice,NaT,NaT,9 days 23:54:55.516952,NaT,NaT,NaT,NaT,NaT,NaT,2020-06-07 22:00:00,NaT,NaN,2020-05-29 08:12:30.061672,NaN,NaN,NaT,NaT,nice,2020-05-29 08:12:30.066381,NaT,NaT,NaT,2020-05,NaT,0,0,21,22-Noche,3,False
2787,2661,0,NaN,10116,1,2020-06-06 16:26:26.030873,NaT,100.0,NaN,money_back,cr_regular,regular,nice,2 days 07:33:33.969127,NaT,31 days 05:33:33.969127,NaT,0 days 00:00:00,NaT,NaT,2020-06-09,2020-07-07 22:00:00.000000,2020-07-07 22:00:00,NaT,NaN,2020-06-06 17:50:34.061784,NaN,NaN,NaT,NaT,nice,2020-12-18 13:08:42.442739,NaT,NaT,NaT,2020-06,2020-06-09,0,1,14,16-Tarde,5,True
8860,8571,0,NaN,10116,1,2020-07-16 22:19:23.191939,NaT,100.0,NaN,rejected,cr_regular,regular,nice,NaT,NaT,19 days 23:40:36.808061,NaT,NaT,NaT,NaT,NaT,NaT,2020-08-05 22:00:00,NaT,NaN,2020-07-17 15:19:10.535343,NaN,NaN,NaT,NaT,nice,2020-07-17 15:19:10.537775,NaT,NaT,NaT,2020-07,NaT,0,0,21,22-Noche,3,False
10023,9689,0,NaN,10116,1,2020-07-25 14:54:32.584620,NaT,100.0,NaN,money_back,cr_regular,regular,nice,2 days 09:05:27.415380,-5 days +09:05:27.415740,11 days 07:05:27.415380,4 days 07:05:27.415740,0 days 00:00:00,6 days 23:59:59.999640,2020-08-01 14:54:32.584260,2020-07-28,2020-08-05 22:00:00.000000,2020-08-05 22:00:00,NaT,NaN,2020-07-25 18:22:07.431781,NaN,NaN,NaT,NaT,nice,2020-12-18 13:09:59.442626,NaT,NaT,NaT,2020-07,2020-07-28,0,2,14,14-Tarde,5,True
12652,11710,3308,11710.0,10116,1,2020-08-10 13:12:53.488655,2020-08-10 16:06:33.831551,50.0,5.0,money_back,accepted,instant,instant_payment,1 days 10:47:06.511345,-6 days +10:47:06.511747,25 days 08:47:06.511345,18 days 08:47:06.511747,-1 days +02:09:33.756680,6 days 23:59:59.999598,2020-08-17 13:12:53.488253,2020-08-12,2020-09-05 19:50:26.243320,2020-09-04 22:00:00,2020-09-05 19:50:26.278501,after,2020-08-10 16:06:32.371677,Instant Payment Cash Request 11710,nice,NaT,NaT,nice,2020-12-18 13:10:36.379843,NaT,NaT,2020-10-13 14:25:05.224806,2020-08,2020-08-12,1,3,7,13-Mañana,0,True
19030,16298,7855,16298.0,10116,1,2020-09-15 19:11:29.644998,2020-09-16 07:55:35.427674,100.0,5.0,money_back,accepted,instant,instant_payment,1 days 04:48:30.355002,-6 days +04:48:30.403660,20 days 02:48:30.355002,13 days 02:48:30.403660,-1 days +02:26:06.846194,6 days 23:59:59.951342,2020-09-22 19:11:29.596340,2020-09-17,2020-10-06 19:33:53.153806,2020-10-05 22:00:00,2020-10-06 19:33:53.180290,after,2020-09-16 07:55:34.095459,Instant Payment Cash Request 16298,nice,NaT,NaT,nice,2020-12-18 13:10:33.519498,NaT,NaT,2020-10-13 14:25:11.725538,2020-09,2020-09-17,2,4,14,19-Tarde,1,True
24303,20112,12214,20112.0,10116,1,2020-10-09 11:12:35.190378,2020-10-09 11:12:45.530120,100.0,5.0,money_back,accepted,instant,instant_payment,0 days 12:47:24.809622,0 days 12:47:18.150094,27 days 10:47:24.809622,27 days 10:47:18.150094,-1 days +01:24:56.222555,0 days 00:00:06.659528,2020-10-09 11:12:41.849906,2020-10-10,2020-11-06 20:35:03.777445,2020-11-05 22:00:00,2020-11-06 20:35:03.804527,after,NaT,Instant Payment Cash Request 20112,nice,NaT,NaT,nice,2020-12-18 13:08:29.865140,NaT,NaT,2020-10-13 14:25:16.972640,2020-10,2020-10-10,3,5,7,11-Mañana,4,True


In [10]:
df_jo[df_jo['user_id']==19655]

,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status,updated_at,reco_creation,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,created_at_slot,created_at_slot_h,created_at_dow,good_user
5023,4686,0,NaN,19655,1,2020-06-22 17:40:03.412270,NaT,100.0,NaN,rejected,NaN,regular,nice,NaT,NaT,14 days 04:19:56.587730,NaT,NaT,NaT,NaT,NaT,NaT,2020-07-06 22:00:00.000,NaT,NaN,2020-06-23 10:29:41.571592,NaN,NaN,NaT,NaT,nice,2020-06-30 13:59:11.962802,NaT,NaT,NaT,2020-06,NaT,0,0,14,17-Tarde,0,False
6004,5985,0,NaN,19655,1,2020-06-30 15:22:34.822037,NaT,100.0,NaN,money_back,NaN,regular,nice,1 days 08:37:25.177963,NaT,35 days 06:37:25.177963,NaT,0 days 00:00:00,NaT,NaT,2020-07-02,2020-08-04 22:00:00.000000,2020-08-04 22:00:00.000,NaT,NaN,2020-06-30 16:55:43.660639,NaN,NaN,NaT,NaT,nice,2020-12-18 13:09:17.207778,NaT,NaT,NaT,2020-06,2020-07-02,0,1,14,15-Tarde,1,False
18501,15904,0,NaN,19655,1,2020-09-12 19:23:14.091080,NaT,100.0,NaN,money_back,NaN,regular,nice,9 days 04:36:45.908920,2 days 04:36:45.952007,24 days 02:36:45.908920,17 days 02:36:45.952007,-1 days +02:02:25.446652,6 days 23:59:59.956913,2020-09-19 19:23:14.047993,2020-09-22,2020-10-07 19:57:34.553348,2020-10-06 22:00:00.000,NaT,NaN,2020-09-13 11:30:46.611801,NaN,NaN,NaT,NaT,nice,2020-12-18 13:11:32.891243,NaT,NaT,NaT,2020-09,2020-09-22,0,2,14,19-Tarde,5,False
24822,20512,0,NaN,19655,1,2020-10-11 21:26:49.118189,NaT,100.0,NaN,money_back,NaN,regular,nice,8 days 02:33:10.881811,1 days 02:33:10.884009,14 days 23:56:10.378811,7 days 23:56:10.381009,-4 days +22:22:59.497000,6 days 23:59:59.997802,2020-10-18 21:26:49.115991,2020-10-20,2020-10-29 23:00:00.000000,2020-10-26 21:22:59.497,NaT,NaN,NaT,NaN,NaN,NaT,NaT,nice,2020-12-18 13:10:09.153745,NaT,NaT,NaT,2020-10,2020-10-20,0,3,21,21-Noche,6,False
